# New York taxi duration 

이 경진대회는 많은 분들이 캐글 초보 커리큘럼에서 추천하고 있는 경진대회 이기에 저 또한 다른 커널들을 읽어보고 스스로 생각해본바를 적어보았습니다. 모델링과 좋은 점수에 초점을 두지 않고 통계적 지식을 이용한 탐색적 데이터 분석에 초점을 두고 노트를 작성해보았습니다. 



## 목차

1. [데이터 파악 및 전처리](#전처리)  
   
   1.1 [feature를 보고 드는 생각](#opinion)
   
   1.2 [feature engineering](#feature_engineering)
   
2. [EDA](#EDA)

   2.1 [target 분포](#target_distribution) 
   
   2.2 [feature 와 target의 관계 시각화 및 통계 요약치](#feature_val) 

## 1. 데이터 파악 및 전처리 
<a id='전처리'></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import skew, norm
from sklearn.cluster import KMeans
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/train.zip')
test  = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/test.zip')
sub   = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip')

In [ ]:
# target 인 trip duration과 target을 계산 할 수 있는 dropoff time 이 없다. 
print(train.shape)
print(test.shape)

In [ ]:
train.info()

In [ ]:
train.head()

- 날짜 데이터 분리

In [ ]:
def datetime_split(data) :
    data['pickup_datetime'] = data.pickup_datetime.apply(pd.to_datetime)
    data['year'] = data.pickup_datetime.apply(lambda x : x.year)
    data['month'] = data.pickup_datetime.apply(lambda x : x.month)
    data['day'] = data.pickup_datetime.apply(lambda x : x.day)
    data['hour'] = data.pickup_datetime.apply(lambda x  : x.hour)
    data['dayofweek'] = data.pickup_datetime.apply(lambda x : x.dayofweek)
    
datetime_split(train)
datetime_split(test)

In [ ]:
train.head()

In [ ]:
train.year.value_counts() 

train.drop('year', axis = 1,  inplace = True)
test.drop('year', axis = 1, inplace = True)

## 1.1 feature 확인 후 드는 생각 
<a id='opinion'></a>
 
 - pickup_datetime 을 살펴보면 모두 2016년 데이터 이므로 year는 삭제 하였다. 시간별로 택시 이용시간이 다를것으로 예상이 된다. 또한 공휴일과 아닌날의 차이가 있을것이라 생각이든다. 요일별로 나눠보기도 하고 주말 or not 의 형태로 나눠봐야될거 같다. 월별로 택시 이용시간이 다를수 있다. 왜냐면 서울로 생각을 해보면 봄에 여의도 부근은 벚꽃축제가 열리기 때문에 평소보다 더 많이 막히게 되고 택시를 이용할시 시간이 더 오래 걸리기 때문에 특정 시간대에 따라 교통상황이 변할 수 있기 때문에 target이 달라질수 있지 않을까 생각이 든다. 
 - passenger_count를 살펴 보았을때 승객의 수가 택시의 승차 시간에 영향을 줄지 궁금하다. 
 - 승차 위치와 하차 위치가 데이터로 존재하는데 직관적으로 생각 나는 것은 승차위치와 하차 위치로 움직인 거리를 도출 하면 상관계수가 꽤나 높지 않을까 생각이 들었다. 먼거리 일수록 오래타야되는 것은 많으니까 
 - 하지만 서울 강남에서 1km 이동하는 시간이랑 한적한 시골에서1km 이동하는 시간은 다를 것이다. 또한 출퇴근시간이나 사람이 붐비는 특정 시간대라면 같은 장소라 하더라도 이동 시간이 달라질 것이다. 
 - 그러므로 시간대, 위치 와 도출한 거리의 관계를 좀 살펴봐야될거 같다. 
 - store_and_fwd_flag는 무슨 뜻인지 잘 모르겠으나 일단 나눠서 판단해 봐야 되겠다. 뭐..저장을 어쩌고 하던데 잘모르겠다..

## 1.2 Feature engineering

<a id='feature_engineering'></a>

### 승차 위치와 하차 위치를 통해 움직인 거리를 구한다.
- [Link](#https://www.kaggle.com/karelrv/nyct-from-a-to-z-with-xgboost-tutorial) 에서 만들어주신 거리 계산 함수를 이용해 보았다.  
- 거리의 종류는 여러가지다. 여기서는 택시가 움직인 거리가 필요 하므로 유클리디안 거리와 맨해튼 거리를 구해보려 한다. 
- 유클리디안 거리란 두 점의 공간 차이의 제곱합을 말하고 맨해튼 거리는 두 점의 공간 차이의 절대값의 합을 말한다.
- 맨해튼 거리는 택시거리라고도 불리기에 이번 택시 이용시간 예측에 가장 적합한 변수라 생각이 든다. 

In [ ]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

train['haversine'] = haversine_array(train['pickup_latitude'], 
                                     train['pickup_longitude'],
                                     train['dropoff_latitude'],
                                     train['dropoff_longitude'])

train['manhattan'] = dummy_manhattan_distance(train['pickup_latitude'], 
                                     train['pickup_longitude'],
                                     train['dropoff_latitude'],
                                     train['dropoff_longitude'])
test['haversine'] = haversine_array(test['pickup_latitude'], 
                                     test['pickup_longitude'],
                                     test['dropoff_latitude'],
                                     test['dropoff_longitude'])

test['manhattan'] = dummy_manhattan_distance(test['pickup_latitude'], 
                                     test['pickup_longitude'],
                                     test['dropoff_latitude'],
                                     test['dropoff_longitude'])


In [ ]:
train.head()

### 승하차 지역


- new york의 행정구역은 총 200개다. 
- new york도 서울처럼 구별로 유동인구의 분포가 다를 것이라는 가정하에 분석을 진행해보았다.
- 이 데이터로는 승하차의 위도 경도정보만 알 수 있고 지역의 이름은 알 수 없다. 
- 각 지역별 측정 위치를  파악한 뒤  실제 데이터와 측정위치의 거리 차이를 이용해 실제데이터가 어느 지역에 속해 있는지 판단할 경우 계산량이 너무 많아진다는 단점이있다.
- 따라서 비지도 학습인 kmeans를 이용해 지역 변수를 얻어 내고자 한다. 
- 판단에 오류가 생길 수도 있으나 대략적인 분석을 위해 이와 같은 분석기법을 이용하였다.

In [ ]:
xlim = [-74.03, -73.77] 
ylim = [40.63, 40.85]

In [ ]:
plt.scatter(train['pickup_longitude'].values[:100000], train['pickup_latitude'].values[:100000],
              color='blue', s=1, label='train', alpha=0.1)
plt.xlim(xlim)
plt.ylim(ylim)

In [ ]:
loc_df = pd.DataFrame()
longitude = list(train.pickup_longitude) + list(train.dropoff_longitude)
latitude = list(train.pickup_latitude) + list(train.dropoff_latitude)
loc_df['longitude'] = longitude
loc_df['latitude'] = latitude


kmeans = KMeans(n_clusters=12, random_state=2, n_init = 20).fit(loc_df)

loc_df['label'] = kmeans.labels_

loc_df = loc_df.sample(200000)
plt.figure(figsize = (10,10))
for label in loc_df.label.unique():
    plt.plot(loc_df.longitude[loc_df.label == label],loc_df.latitude[loc_df.label == label],'.', alpha = 0.3, markersize = 0.3)

plt.xlim(xlim)
plt.ylim(ylim)
plt.title('Clusters of New York')
plt.show()

# 2.EDA
<a id='EDA'></a>

## 2. 1 target 분포 확인
<a id='target_distribution'></a>

In [ ]:
target = train['trip_duration']

sns.distplot(target, fit = norm)

plt.xlabel('Trip_duration')

- 분포의 모양이 이상하다. 이상치로 인해 이런 모양을 띄는거 같으므로 boxplot을 그려본다.

In [ ]:
sns.boxplot(target)

- 4개의 점이 이상하게 많이 동떨어져 있다.
- 4개만 따로 확인해봐야 될거 같다.

In [ ]:
train[train['trip_duration'] > 1500000]

- 승차 시간이랑 하차시간의 날짜가 다르다... 첫번째 행을 보면 22일동안 택시를 탔다는 뜻인데 이러한 이상치는 아마 택시 미터기를 끄지 않은 것이 아닐까? 혹은 미터기의 고장으로 인해 잘못된 데이터일 가능성이 높다. 상식적으로 하루이상 택시를 타는것도 이상한데 몇주 동안 택시를 탄다는 것은 말이 되지 않다. 그러므로 이 행들은 삭제를 하겠다.

In [ ]:
train.drop(train[train['trip_duration'] > 150000].index, axis = 0, inplace = True)

### 이상치 제거 후 종속변수 분포 확인

In [ ]:
sns.distplot(train['trip_duration'], fit = norm)

plt.xlabel('Trip_duration')

In [ ]:
train[train['trip_duration'] >= 20000]

- 여전히 승차시간이 24시간이나 되는 데이터들이 존재 한다. 
- 이는 데이터가 잘못됐다고 생각이 들어 이상치를 제거해주도록 하겠다.

In [ ]:
Q1 = np.percentile(train['trip_duration'], 25) 
Q3 = np.percentile(train['trip_duration'], 75)
IQR = Q3 - Q1 
outlier_step = 1.5 * IQR 
outlier_list_idx = train[train['trip_duration'] > Q3 + outlier_step].index

train.drop(outlier_list_idx, axis = 0, inplace = True)

In [ ]:
sns.distplot(train['trip_duration'], fit = norm)

plt.title('skewness of target {0:.3f}'.format(skew(train['trip_duration'])))

- boxplot에서 보이는 이상치들을 제거 했다. 
- 값이 과하게 큰 값을 가지는 데이터만이 이상치라고 생각을 했다. 
- 3/4 분위수와 1/4분위스의 차이를 iqr(Inter - Quatitle - range)이라하는데 이 iqr의 1.5배 만큼 떨어져있는 데이터를 이상치로 여길수 있다.  
- 제거한 이후의 종속변수의 분포는 예전 보다 정규분포와 더 근사한 모습을 보이게 됐다. 
- 하지만 왼쪽으로 좀 치우쳐진 모습을 보이는 로그정규분포의 모습을 보이는거 같아 로그를 취해봤다.

In [ ]:
sns.distplot(np.log1p(train['trip_duration']), fit = norm)

plt.xlabel('trip_duration')
plt.title('skewness of target {0:.3f}'.format(skew(np.log1p(train['trip_duration']))))

조금 더 정규분포에 근사한 모습을 보인다.
따라서 target에 log를 취해주었다.

In [ ]:
ln_target = np.log1p(train['trip_duration'])
train['ln_target'] = ln_target

## 2.2 설명 변수와 종속변수 간의 관계 
<a id='feature_val'></a>

### 거리와 target의 관계 파악


In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)

axes[0].scatter(x = train['haversine'],
                y = train['trip_duration'])
axes[0].set_title('Target ~ haversine distance')
axes[0].set_xlabel('haversine distance')

axes[1].scatter(x = train['manhattan'],
                y = train['trip_duration'], )
axes[1].set_title('Target ~ manhattan distance')
axes[1].set_xlabel('manhattan distance')

plt.tight_layout()
plt.show()

-  예상했던거와 달리 선형관계가 뚜렷해 보이지는 않는다. 
-  거리의 기준이 km 인데 500km 를 넘게 이동한 데이터도 확인 할 수 있다. 얼마나 돈이 많길래 500km가 넘는 거리를(서울에서 부산까지가 400km정도 한다는데...) 택시를 타고 다니는것 일까  
- 아마 잘못 기록된것이 아닐까 생각이 든다. 하지만 testset에서도 이러한 이상치로 예상이 되는 데이터가 존재하는지 확인해봐야된다.

In [ ]:
sns.boxplot(test['haversine'])

plt.xlabel('haversine')
plt.title('Boxplot of haversine')

- 혹시나가 역시나 test set에도 이상치로 여겨지는 데이터들이 존재한다. 이럴경우 이 데이터들을 제외하고 예측 할 수 없으므로 이상치의 힘을 약화 시킬 무기가 필요하다. 

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=False)

axes[0].boxplot(train['haversine'])
axes[0].set_title('haversine distance \n skewness {:.3f}'.format(skew(train['haversine'])))
axes[0].set_xlabel('haversine distance')

axes[1].boxplot(np.log1p(train['haversine']))
axes[1].set_title('Log haversine distance \n skewness {:.3f}'.format(skew(np.log1p(train['haversine']))))
axes[1].set_xlabel('log haversine distance')


plt.tight_layout()
plt.show()

- Log를 취했을 때 이상치는 줄어들었다고 말할 수 는 없지만 그래도 box의 모양이 뚜렷해 졌다. 
- 분포의 왜도(skewness) 또한 약 65에서 약 0.76으로 줄어 들었다. 참고로 왜도는 분포의 3차 적률과 연관이 되어있는데 정규분포의 경우 왜도가 0 이며 정규분포에 근사하다고 말할수있는 기준은 절대값 2이다. 왜도 값이 0이라는 것은 해당 분포가 완벽한 좌우 대칭의 형태를 보인다는 뜻이다. 
- 왜도가 0에 가까울 수록 ols 추정법에서 세우는 가정 중 하나인 "모든 변수의 분포는 정규분포를 따른다."를 만족 할수 있다. 

### log를 취한 거리 변수 생성 및 target과의 관계

In [ ]:
snfig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
train['ln_haversine'] = np.log1p(train['haversine'])
train['ln_manhattan'] = np.log1p(train['manhattan'])
test['ln_haversine'] = np.log1p(test['haversine'])
test['ln_manhattan'] = np.log1p(test['manhattan'])

axes[0].scatter(x = train['ln_haversine'],
                y = train['trip_duration'])
axes[0].set_title('Target ~ Log haversine distance \n correlation : {:.3f}'.format(train[['ln_haversine','trip_duration']].corr().iloc[1,0]))
axes[0].set_xlabel('Log haversine distance')

axes[1].scatter(x = train['ln_manhattan'],
                y = train['trip_duration'])
axes[1].set_title('Target ~ Log manhattan distance \n correlation : {:.3f}'.format(train[['ln_manhattan','trip_duration']].corr().iloc[1,0]))
axes[1].set_xlabel('Log manhattan distance')

plt.tight_layout()
plt.show()

In [ ]:
snfig, axes = plt.subplots(1, 2, sharex=True, sharey=True)

axes[0].scatter(x = train['ln_haversine'],
                y = train['ln_target'])
axes[0].set_title('Log Target ~ Log haversine distance\n correlation : {:.3f}'.format(train[['ln_haversine','ln_target']].corr().iloc[1,0]))
axes[0].set_xlabel('Log haversine distance')

axes[1].scatter(x = train['ln_manhattan'],
                y = train['ln_target'])
axes[1].set_title('Log Target ~ Log manhattan distance \n correlation : {:.3f}'.format(train[['ln_manhattan','ln_target']].corr().iloc[1,0]))
axes[1].set_xlabel('Log manhattan distance')

plt.tight_layout()
plt.show()

- 두 변수 모두 log를 취했을 때 조금 더 체계적인 관계를 보인다.
- 상관계수를 통해 이를 확인해보겠다.

In [ ]:
train[['haversine', 'manhattan','ln_haversine','ln_manhattan','trip_duration','ln_target']].corr()

- 거리에 log를 취했을 때 종속변수와의 상관계수가 더 큰 것을 확인했다. 
- 다만 종속변수에 log를 취하면 취하기전보다 더 낮은 상관계수를 가지는데 이는 두 변수다 log를 취했을 때 산점도를 보면 비선형관계에 더 가까워 보이기 떄문이 아닐까 싶다. (2차 다항식) 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
# linear model 
ln_target = train['ln_target']
lr_m = LinearRegression()
poly_m = LinearRegression()

train_distance = train.ln_haversine.values

train_distance = train_distance.reshape(-1, 1)

poly = PolynomialFeatures(degree=2)

train_dist_sqr = poly.fit_transform(train_distance)

lr_m.fit(train_distance, ln_target)
poly_m.fit(train_dist_sqr, ln_target)

pred_lr = lr_m.predict(train_distance)
pred_poly = poly_m.predict(train_dist_sqr)

lr_score = mean_squared_error(ln_target, pred_lr)
poly_score = mean_squared_error(ln_target, pred_poly)

r2_lr = r2_score(ln_target, pred_lr)
r2_poly = r2_score(ln_target, pred_poly)

print('r2_score (degree = 1) : {0:.3f} \n  MSE : {1:.3f}'.format(r2_lr, lr_score))
print('======================================================')
print('r2_score(degree = 2) : {0:.3f} \n MSE : {1:.3f}'.format(r2_poly, poly_score))
print('======================================================')
print('polynomial regression estimators : ({1:.3f}) * hour^2 + ({0:.3f}) * hour + ({2:.3f})'
      .format(poly_m.coef_[1],poly_m.coef_[2], poly_m.intercept_))

- 거리를 2차항으로 변환하여 선형회귀분석을 한 결과가 1차항일때 보다 더 나은 지표를 보인다.
- mse와 r^2가 좋다고해서 좋은 예측력이 보장이 되는것은 아니다. 오버피팅의 문제가 있을수 있기 떄문이다. 
- 2차항을 변환했을 때 지표가 좋아지기는 했으나 그 폭이 작으므로 다항식으로 변환하는것은 좋아 보이지 않는다.

### 시간대 별 target boxplot

In [ ]:
sns.boxplot(x = train['hour'],
            y = train['ln_target'])


plt.title('Log trip duration ~ Hour')

In [ ]:
sns.boxplot(x = train['dayofweek'],
            y = train['ln_target'])

plt.title('Log Target ~ Dayofweek')

- 시간과 요일별 target의 분포의 차이는 눈에 띌 정도로 보이지 않는다.
- 평일과  주말을 나눠서 시간대 별로 살펴 보려한다.

#### 주말 변수 생성 (주말이면 1, 아니면 0)

In [ ]:
weekend = pd.DataFrame({'dayofweek' : [0,1,2,3,4,5,6], 'weekend' : [1,0,0,0,0,0,1]})

train = pd.merge(train, weekend, on = 'dayofweek')
test = pd.merge(test, weekend, on = 'dayofweek')


In [ ]:
train.head()

### 주말별 target 분포 확인

In [ ]:
sns.boxplot(x = train['weekend'],
            y = train['ln_target'])

In [ ]:
train.groupby(by = 'weekend').boxplot(column = ['ln_target'], by = ['hour'], figsize = (20,20))

- 별차이가 없어 보인다. 

### 탑승 승객 별 target 분포 확인

In [ ]:
sns.boxplot(x = train['passenger_count'],
            y = train['ln_target'])

- passenget_count가 0인 값을가지는 이상한 데이터가 존재하는 것을 확인했다. 유령이 타지 않는한 승객이 0명일리는 없을 것이다. 결측치로 여기고 적당한 값으로 대체를 해야겠다.
- 그리고 7인이상 탑승한 데이터는 모두 6으로 바꿔주는것이 나을거 같다. 
- 승객이 1~6명일때 target의 boxplot 뚜렷한 차이점을 보여주지 않는다. 

In [ ]:
print(train.passenger_count.value_counts())
print('==============================')
print(test.passenger_count.value_counts())

In [ ]:
train['passenger_count'].replace(0,1, inplace= True)
train['passenger_count'].replace(7,6, inplace= True)
train['passenger_count'].replace(8,6, inplace= True)
train['passenger_count'].replace(9,6, inplace= True)

test['passenger_count'].replace(0,1, inplace= True)
test['passenger_count'].replace(9,6, inplace= True)


In [ ]:
train.passenger_count.value_counts()

In [ ]:
sns.boxplot(x = train['passenger_count'],
            y = train['ln_target'])

- 승객의 수 별로 target의 boxplot이 다른 모습을 보이지 않는다. 

###  store_and_fwd_flag와 target의 관계 

In [ ]:
sns.boxplot(x = train['store_and_fwd_flag'],
            y = train['ln_target'])

plt.title('Log Target ~ store and fwd flag')

- N일 경우 이상치의 꼬리가 더 길어 보이지만 큰 차이는 없어 보인다

### EDA 결과 

위도경도로 얻은 거리외에 다른 변수들은 target에 큰 영향을 주지 못할것으로 예상이 된다. 